#                     <font color=red>       BBM409 : Introduction to Machine Learning Lab  Assignment 2</font>

<img src="logo.png" width=300 height=200 />

## <font color=black><center>Berra Nur SARI - 21727671 <br> Melih SUNMAN - 21827809</center></font>

#   <font color=blue>Part 1: Diabetes Risk Prediction
</font>


## Abstract :

In the first part of this project, we use ID3 algorithm for creating desicion tree. This tree analyzes a dataset of a group of people to assess whether they are at risk for diabetes. We used 5-Fold Cross Validation for better processing of data and also discretization has been applied to the age data to make the ID3 algorithm work better.

## Code :

#### Required libraries are imported 

In [930]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score ,precision_score , f1_score , recall_score
from sklearn.model_selection import KFold
import time

#### Necessary variables were created for the analysis of the data

In [931]:
#upload file
df = pd.read_csv("diabetes_data_upload.csv")

#rename the last column name to label
df = df.rename(columns={"class": "label"})

In [932]:
global COLUMN_HEADERS
COLUMN_HEADERS = list(df.columns.values.tolist())

global COLUMN_IDS
COLUMN_IDS = np.arange(len(COLUMN_HEADERS))

In [933]:
#convert from dataframe to numpy
df = df.to_numpy()

#shuffle
np.random.shuffle(df)

#### check_purity(data) : 
Function that checks if the entire dataset is classified. If classification is finished, terminates branching continues if not. It returns False because **df** hasn't classified yet


In [934]:
def check_purity(data):
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)
    if len(unique_classes ) == 1:
        return True;
    return False

In [935]:
check_purity(df)

False

#### classify_data(data) : 
Function that determines which label is more in the data set. **df** returns Positive in this function because there are more Positives in it

In [936]:
def classify_data(data):
    label_column = data[:,-1]
    #unique labellar arasından en çok olanı döndürüyoruz, yani count ı max olanı
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
    max_index = counts_unique_classes.argmax()
    classification = unique_classes[max_index]
    return classification

In [937]:
classify_data(df)

'Positive'

#### age_splits(data, column_index, split_number):

Function that performs the discretization we need for the age variable. It takes dataset, column number and discretization number as parameters

In [938]:
def age_splits(data, column_index, split_number):
    gap = (data[:, column_index]).max() - (data[:, column_index]).min()
    average = gap/split_number
    ranges = data[:, column_index].min()
    age_ranges = []
    while ranges <= data[:, column_index].max():
        age_ranges += [int(ranges)]
        ranges += average
    #print(age_ranges)
    for j in range(len(data[:, column_index])):
        for i in range(len(age_ranges) - 1):
            if data[:, column_index][j] <= age_ranges[i + 1] and data[:, column_index][j] >= age_ranges[i]:
                #print(data[:, column_name][j], "is belong to Range {}".format(i+1))
                data[:, column_index][j] = i + 1
    return data
df = age_splits(df,0,5)

In [939]:
df

array([[3, 'Male', 'No', ..., 'Yes', 'Yes', 'Positive'],
       [1, 'Male', 'No', ..., 'No', 'No', 'Negative'],
       [4, 'Male', 'No', ..., 'Yes', 'No', 'Negative'],
       ...,
       [2, 'Female', 'No', ..., 'No', 'No', 'Positive'],
       [4, 'Male', 'Yes', ..., 'Yes', 'No', 'Positive'],
       [2, 'Female', 'No', ..., 'No', 'No', 'Negative']], dtype=object)

#### get_potential_splits(data, attribute_ids):
function that returns the values of the rows, thus determining which branches the decision tree will have. It takes dataset and column Id that values needed to know as parameters.

In [940]:
def get_potential_splits(data, attribute_ids):
    potential_splits = {}
    for column_index in attribute_ids:  #excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        potential_splits[COLUMN_HEADERS[column_index]] = unique_values
    return potential_splits

In [941]:
get_potential_splits(df, COLUMN_IDS)

{'Age': array([1, 2, 3, 4, 5], dtype=object),
 'Gender': array(['Female', 'Male'], dtype=object),
 'Polyuria': array(['No', 'Yes'], dtype=object),
 'Polydipsia': array(['No', 'Yes'], dtype=object),
 'sudden weight loss': array(['No', 'Yes'], dtype=object),
 'weakness': array(['No', 'Yes'], dtype=object),
 'Polyphagia': array(['No', 'Yes'], dtype=object),
 'Genital thrush': array(['No', 'Yes'], dtype=object),
 'visual blurring': array(['No', 'Yes'], dtype=object),
 'Itching': array(['No', 'Yes'], dtype=object),
 'Irritability': array(['No', 'Yes'], dtype=object),
 'delayed healing': array(['No', 'Yes'], dtype=object),
 'partial paresis': array(['No', 'Yes'], dtype=object),
 'muscle stiffness': array(['No', 'Yes'], dtype=object),
 'Alopecia': array(['No', 'Yes'], dtype=object),
 'Obesity': array(['No', 'Yes'], dtype=object),
 'label': array(['Negative', 'Positive'], dtype=object)}

#### calculate_entropy(data):
Function that calculates the ENTROPHY required to calculate the GAIN value. In the calculation, the ENTROPHY value of the data set is high. This shows us that there is a polarized distribution

In [942]:
def calculate_entropy(data):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)
    probabilities = counts / counts.sum()  
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [943]:
calculate_entropy(df)

0.9612366047228759

#### calculate_gain(data, index):
function that returns the GAIN value, which allows us to decide on the next branch of the decision tree. It takes dataset and column index as parameter. In the calculation, we see the GAIN value of the **age** attribute.

In [944]:
#calculate entropy for determined column
def calculate_gain(data, index):
    values, counts = np.unique(data[:,index], return_counts=True)
    total = sum(counts)
    full_entropy = calculate_entropy(data)
    for i in range(len(values)):
        value_data = data[data[:,index] == values[i]]
        value_entropy = calculate_entropy(value_data)
        full_entropy -= (counts[i]/total) * value_entropy
    return full_entropy

In [945]:
calculate_gain(df,0)

0.03049821131260183

#### def calculate_best_gain(data, attribute_ids):
Function that returns the best GAIN value in a given dataset and attribute that has this value. It takes dataset and ID list of attributes as parameter. In this example we see the GAIN value = 0.3622... and index 2 = Polyuria

In [946]:
def calculate_best_gain(data, attribute_ids):
    gains = {}
    for id in attribute_ids:
        gain = calculate_gain(data, id)
        gains[id] = gain

    best_gain_key = max(gains, key=gains.get)
    return gains[best_gain_key], best_gain_key

In [947]:
calculate_best_gain(df, COLUMN_IDS[:-1])

(0.36225097556392627, 2)

### class Node:
The class in which we create the decision tree <br>
**children** : list of sub branch <br>
**vals** : the value list that attribute has (for age = 1,2,3,4,5 for polyuria = yes, no ... ) <br>
**rule** : rule of desicion tree <br>
**max_gain** : GAIN of that branch <br>
**attribute_name** : Name of attribute (Age,Gender,Polyuria,Polydipsia,sudden weight loss
...) <br>
**attribute_id** : Index of the attribute (for age = 0 , gender = 1, Polyuria = 2 ...)

In [948]:
class Node:
    def __init__(self, attribute_id, attribute_name, vals, rule):
        self.children = []
        self.vals = vals
        self.rule = rule
        self.max_gain = None
        self.attribute_name = attribute_name
        self.attribute_id = attribute_id

In [949]:
global POTENTIAL_SPLITS
POTENTIAL_SPLITS = get_potential_splits(df,COLUMN_IDS)

In [950]:
POTENTIAL_SPLITS

{'Age': array([1, 2, 3, 4, 5], dtype=object),
 'Gender': array(['Female', 'Male'], dtype=object),
 'Polyuria': array(['No', 'Yes'], dtype=object),
 'Polydipsia': array(['No', 'Yes'], dtype=object),
 'sudden weight loss': array(['No', 'Yes'], dtype=object),
 'weakness': array(['No', 'Yes'], dtype=object),
 'Polyphagia': array(['No', 'Yes'], dtype=object),
 'Genital thrush': array(['No', 'Yes'], dtype=object),
 'visual blurring': array(['No', 'Yes'], dtype=object),
 'Itching': array(['No', 'Yes'], dtype=object),
 'Irritability': array(['No', 'Yes'], dtype=object),
 'delayed healing': array(['No', 'Yes'], dtype=object),
 'partial paresis': array(['No', 'Yes'], dtype=object),
 'muscle stiffness': array(['No', 'Yes'], dtype=object),
 'Alopecia': array(['No', 'Yes'], dtype=object),
 'Obesity': array(['No', 'Yes'], dtype=object),
 'label': array(['Negative', 'Positive'], dtype=object)}

## Desicion Tree Algorithm (ID3)
A ID3 decision tree is supervised machine learning algorithm usually used for classification problems. Invented by Ross Quinlan. Decision tree is flow chart like structure used for decision making. Data-set is split in number of iteration in order to create tree. This tree can be viewed as IF-ELSE conditions and we call this Tree Model in Machine Learning. <br>

<img src="expTree.png" width=450 height=300 />


In [951]:
def decision_tree_algorithm(data, attribute_ids, rule):
    """if there is no need to parse the data anymore, the function returns leaf Node(Positive or Negative)"""
    if (check_purity(data)):
        classification = classify_data(data)
        new_rule = str(rule) + classification
        print(new_rule)
        return Node(-1, COLUMN_HEADERS[-1], classification , new_rule) #leaf node
        

        
    """
    The GAIN value of all attributes in the attribute_ids list is calculated and the calculated ones are 
    removed from the list. When the list is completely empty, the variable with the highest GAIN value is
    declared as the new LEAF.
    """
    if (len(attribute_ids[:-1]) == 0):
        targets, counts = np.unique(data[:,-1], return_counts=True)
        index = np.argmax(counts)
        new_rule = str(rule) + targets[index]
        print(new_rule)
        return Node(-1, COLUMN_HEADERS[-1], targets[index], new_rule) #leaf node

    """Determining the attribute with the highest GAIN value (value and index)"""
    best_attr_val, best_attr_index = calculate_best_gain(data, attribute_ids[:-1])

    """Determining the possible values of best attribute (for age 1,2,3,4,5, for Itching yes,no)"""
    potential_values = POTENTIAL_SPLITS[COLUMN_HEADERS[best_attr_index]]

    """Creating tree node"""
    child_node = Node(best_attr_index, COLUMN_HEADERS[best_attr_index], potential_values,rule)
    child_node.max_gain = best_attr_val
    child_node.rule += COLUMN_HEADERS[best_attr_index]  + " -> "
    for val in potential_values: #yes and no or 1 2 3 4 5
        val_data = data[data[:,best_attr_index] == val] #a new matrix is created from those whose best_attr_index is yes

        if val_data.shape[0] == 0:
            targets, counts = np.unique(data[:, -1], return_counts=True)
            index = np.argmax(counts)
            new_rule = str(child_node.rule) + str(val) + " -> " + targets[index]
            print(new_rule)
            child_node.children.append(Node(-1, COLUMN_HEADERS[-1], targets[index],new_rule)) #leaf node
        else:
            attribute_ids_cpy = attribute_ids[attribute_ids != best_attr_index]
            new_rule = str(child_node.rule) + str(val) + " -> "
            child_node.children.append(decision_tree_algorithm(val_data, attribute_ids_cpy ,new_rule)) #recursive

    return child_node  #root node


#### predict_row_recr(node, row) and predict(root, data):
Functions that make predictions by evaluating the given test class in the current decision tree. Working synchronously, they classify each variable individually. First, the *predict(root, data)* function takes as parameters the current decision tree and the dataset to be tested. It loops through the dataset and sends a given row to the *predict_row_recr(node, row)* function. This function evaluates the row in with the decision tree and returns to the *predict(root, data)* function. The predictions are aggregated in the *predicts[ ]* list and the list is returned when all rows are finished


In [952]:
def predict_row_recr(node, row):
    if node.attribute_name == 'label':
        return node.vals
    else:
        #print(node.vals , " ", row[node.attribute_id]) #test row unun x attribute-undeki değer ne, mesela yes, o zaman yes e karşılık gelen attr yeni nodu un olacak.
        index = np.nonzero(node.vals == row[node.attribute_id])[0][0]
        next_node = node.children[index]
        return predict_row_recr(next_node, row)

In [953]:
def predict(root, data):
    predicts = []
    for row in data:
        x = predict_row_recr(root, row)
        predicts.append(x)
    return predicts

#### 5-Fold Cross Validation 
At the last part we used 5-Fold Cross Validation getting more valid scores. Then we print the **ACCURACY, PRECISION, RECALL** and **F1 SCORE**

In [954]:
k_fold = KFold(n_splits=5, shuffle=True)
c = 0
for train_index, test_index in k_fold.split(df):
    startTime = time.time()
    train_data , test_data =  df[train_index], df[test_index]
    tree = decision_tree_algorithm(train_data, COLUMN_IDS , "")
    predicts = predict(tree, test_data)
    c+=1
    print("ACCURACY FOR " , c , end = " ")
    print(accuracy_score(test_data[:,-1], predicts))

    print("PRECISION FOR " , c , end = " ")
    print(precision_score(test_data[:,-1], predicts, average="binary" , pos_label="Positive"))

    
    print("RECALL FOR " , c , end = " ")
    print(recall_score(test_data[:,-1], predicts ,average="binary", pos_label="Positive"))

    print("F1 SCORE FOR " , c , end = " ")
    print(f1_score(test_data[:,-1], predicts,average="binary",pos_label="Positive"))
    endTime = time.time()
    print("Computation time:", (endTime-startTime))


Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> Alopecia -> No -> Age -> 1 -> Polyphagia -> No -> Positive
Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> Alopecia -> No -> Age -> 1 -> Polyphagia -> Yes -> Negative
Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> Alopecia -> No -> Age -> 2 -> visual blurring -> No -> Irritability -> No -> weakness -> No -> sudden weight loss -> No -> Obesity -> No -> Polyphagia -> No -> Genital thrush -> No -> Itching -> No -> delayed healing -> No -> partial paresis -> No -> muscle stiffness -> No -> Positive
Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> Alopecia -> No -> Age -> 2 -> visual blurring -> No -> Irritability -> No -> weakness -> No -> sudden weight loss -> No -> Obesity -> No -> Polyphagia -> No -> Genital thrush -> No -> Itching -> No -> delayed healing -> No -> partial paresis -> No -> muscle stiffness -> Yes -> Positive
Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> Alopecia -> No -> 

Polydipsia -> No -> Gender -> Male -> Polyuria -> No -> Irritability -> No -> weakness -> No -> partial paresis -> No -> Negative
Polydipsia -> No -> Gender -> Male -> Polyuria -> No -> Irritability -> No -> weakness -> No -> partial paresis -> Yes -> Positive
Polydipsia -> No -> Gender -> Male -> Polyuria -> No -> Irritability -> No -> weakness -> Yes -> Age -> 1 -> Negative
Polydipsia -> No -> Gender -> Male -> Polyuria -> No -> Irritability -> No -> weakness -> Yes -> Age -> 2 -> Genital thrush -> No -> Positive
Polydipsia -> No -> Gender -> Male -> Polyuria -> No -> Irritability -> No -> weakness -> Yes -> Age -> 2 -> Genital thrush -> Yes -> Polyphagia -> No -> Negative
Polydipsia -> No -> Gender -> Male -> Polyuria -> No -> Irritability -> No -> weakness -> Yes -> Age -> 2 -> Genital thrush -> Yes -> Polyphagia -> Yes -> Positive
Polydipsia -> No -> Gender -> Male -> Polyuria -> No -> Irritability -> No -> weakness -> Yes -> Age -> 3 -> delayed healing -> No -> visual blurring ->

Polyuria -> Yes -> Polydipsia -> No -> delayed healing -> No -> Positive
Polyuria -> Yes -> Polydipsia -> No -> delayed healing -> Yes -> Itching -> No -> Positive
Polyuria -> Yes -> Polydipsia -> No -> delayed healing -> Yes -> Itching -> Yes -> Alopecia -> No -> Positive
Polyuria -> Yes -> Polydipsia -> No -> delayed healing -> Yes -> Itching -> Yes -> Alopecia -> Yes -> Age -> 1 -> Negative
Polyuria -> Yes -> Polydipsia -> No -> delayed healing -> Yes -> Itching -> Yes -> Alopecia -> Yes -> Age -> 2 -> sudden weight loss -> No -> Positive
Polyuria -> Yes -> Polydipsia -> No -> delayed healing -> Yes -> Itching -> Yes -> Alopecia -> Yes -> Age -> 2 -> sudden weight loss -> Yes -> Negative
Polyuria -> Yes -> Polydipsia -> No -> delayed healing -> Yes -> Itching -> Yes -> Alopecia -> Yes -> Age -> 3 -> Negative
Polyuria -> Yes -> Polydipsia -> No -> delayed healing -> Yes -> Itching -> Yes -> Alopecia -> Yes -> Age -> 4 -> Negative
Polyuria -> Yes -> Polydipsia -> No -> delayed healing

Polyuria -> No -> Polydipsia -> No -> Gender -> Female -> delayed healing -> No -> visual blurring -> No -> Age -> 1 -> Negative
Polyuria -> No -> Polydipsia -> No -> Gender -> Female -> delayed healing -> No -> visual blurring -> No -> Age -> 2 -> sudden weight loss -> No -> weakness -> No -> Polyphagia -> No -> Genital thrush -> No -> Itching -> No -> Irritability -> No -> partial paresis -> No -> muscle stiffness -> No -> Alopecia -> No -> Obesity -> No -> Positive
Polyuria -> No -> Polydipsia -> No -> Gender -> Female -> delayed healing -> No -> visual blurring -> No -> Age -> 2 -> sudden weight loss -> No -> weakness -> No -> Polyphagia -> No -> Genital thrush -> No -> Itching -> No -> Irritability -> No -> partial paresis -> No -> muscle stiffness -> No -> Alopecia -> No -> Obesity -> Yes -> Positive
Polyuria -> No -> Polydipsia -> No -> Gender -> Female -> delayed healing -> No -> visual blurring -> No -> Age -> 2 -> sudden weight loss -> No -> weakness -> No -> Polyphagia -> No

<img src="tablex.png" width=450 height=300 />


## Error Analysis for Classification
- If we check **Accuracy** values at the table we can see the k=1 and k=4 stuations has same values and others has approximate values. This shows us that correct predictions do not chenge much at various K values and the algorithm makes consistent predictions in different situations. 
- The **PRECISION** value gives us the ratio of the true correct values within the correctly predicted values. The fact that the precision value is 1 at K=3 shows us that all the values estimated as positive are correct. In this case the value of false positive is zero. That means all individuals identified as sick are truly sick
- The **RECALL** value is the ratio of true positive predictions to actual positive values. At the table k=4 column has RECALL equals 1. This means the all positive conditions predicted correct. In our example data set this means all sick individual predicted truely . High recall values very important especially medical stuations like our data set example because diagnosing a sick individual as not sick can have serious consequences. 
- Sometimes, separate evaluation of recall and precision values may not yield accurate results. To evaluate these variables together, we need a different metric. The **F1** score is the harmonic mean of Recall and Precision. The main reason for using the **F1** Score value is not to make an incorrect model selection in data sets that are not evenly distributed. Also, the F1 Score is very important to us because we need a measurement metric that will include all error costs, not just False Negative or False Positive. The best f1 scores in the table are in columns K=1 and K=4. From here, we understand that the most healthy distribution is at 1 and 4 K values.
- When we consider the **time** variable, we see different durations in the calculations. The biggest reason for this is the size of the decision trees created when we mix the data. The size of decision trees affects both training and testing times.

## Misclassified Samples
- Below are some misclassified examples from the dataset. When the examples are examined, we see that the features are very close to each other. This indicates that the misclassification is due to Overfitting. Overfitting is a common occurrence in Decision Tree algorithm. Simply put, Overfit is a situation where the algorithm breaks down from Generalization by over-analyzing the trainset. There are different solutions to this problem. The most commonly used solution for the ID3 algorithm is pruning. Pruning doesn't completely fix this problem, but it does its best. Algorithms like ID3 always have some margin of error.

In [983]:
#disclassifieds = [test_data[:, -1] != predicts]

for i in range(len(predicts)):
    if test_data[i, -1] != predicts[i]:
        print(test_data[i,:] , " but it returns " , predicts[i])
        

[1 'Male' 'No' 'No' 'No' 'No' 'No' 'No' 'No' 'No' 'No' 'No' 'No' 'No' 'No'
 'No' 'Negative']  but it returns  Positive
[3 'Male' 'No' 'No' 'No' 'Yes' 'No' 'No' 'No' 'Yes' 'No' 'Yes' 'No' 'No'
 'Yes' 'No' 'Negative']  but it returns  Positive
[2 'Male' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'No' 'No' 'No' 'No' 'No'
 'No' 'No' 'Negative']  but it returns  Positive
[4 'Male' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'No' 'Yes' 'Yes' 'Yes'
 'Yes' 'Yes' 'No' 'Positive']  but it returns  Negative
[4 'Male' 'Yes' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'Yes'
 'Yes' 'Yes' 'Yes' 'Positive']  but it returns  Negative
[3 'Female' 'No' 'No' 'No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No'
 'Yes' 'Yes' 'Yes' 'Negative']  but it returns  Positive
[3 'Male' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No'
 'Yes' 'Yes' 'No' 'Positive']  but it returns  Negative
[4 'Male' 'Yes' 'No' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'Yes'
 'Yes' 'Yes' 'No' 'Negative']  but it returns  Po

#   <font color=blue>Part 2: Pruning Decision Tree
</font>

#### age_splits(data, column_index, split_number):
Function that performs the discretization we need for the age variable. It takes dataset, column number and discretization number as parameters

In [956]:
def ageSplitter(data,ageClass , splitNumber):
    gap = (data[ageClass]).max() - (data[ageClass]).min()
    average = gap/splitNumber
    ranges = data[ageClass].min()
    ageRanges = []
    while ranges<=data[ageClass].max():
        ageRanges += [int(ranges)]
        ranges+= average
    #print(ageRanges)
    for i in range(len(ageRanges)-1):
        for j in range(len(data[ageClass])):
            if data[ageClass][j] <= ageRanges[i + 1] and data[ageClass][j] >= ageRanges[i]:
                #print(data[ageClass][j], "is belong to Randge {}".format(i+1))
                data.at[j, ageClass] = i+1
    return data

In [957]:
# upload file
df = pd.read_csv("diabetes_data_upload.csv")

In [958]:
# rename the last column name to label
df = df.rename(columns={"class": "label"})

In [959]:
df = ageSplitter(df,"Age",5)

In [960]:
global COLUMN_HEADERS
COLUMN_HEADERS = list(df.columns.values.tolist())

In [961]:
COLUMN_HEADERS

['Age',
 'Gender',
 'Polyuria',
 'Polydipsia',
 'sudden weight loss',
 'weakness',
 'Polyphagia',
 'Genital thrush',
 'visual blurring',
 'Itching',
 'Irritability',
 'delayed healing',
 'partial paresis',
 'muscle stiffness',
 'Alopecia',
 'Obesity',
 'label']

In [962]:
global COLUMN_IDS
COLUMN_IDS = np.arange(len(COLUMN_HEADERS))

In [963]:
COLUMN_IDS

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

After shuffle df, we divided it into 3 separate datasets as train, validate and test.

In [964]:
train, validate, test = np.split(df.sample(frac=1), [int(.6 * len(df)), int(.8 * len(df))])

In [965]:
# convert from dataframes to numpy
df = df.to_numpy()
train = train.to_numpy()
test = test.to_numpy()
validate = validate.to_numpy()

#### check_purity(data) : 
Function that checks if the entire dataset is classified. If classification is finished, terminates branching continues if not. It returns False because **df** hasn't classified yet

In [966]:
def check_purity(data):
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)
    if len(unique_classes) == 1:
        return True;
    return False

#### classify_data(data) : 
Function that determines which label is more in the data set. **df** returns Positive in this function because there are more Positives in it

In [967]:
def classify_data(data):
    label_column = data[:, -1]
    # unique labellar arasından en çok olanı döndürüyoruz, yani count ı max olanı
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
    max_index = counts_unique_classes.argmax()
    classification = unique_classes[max_index]
    return classification

#### get_potential_splits(data, attribute_ids):
Function that returns the values of the rows, thus determining which branches the decision tree will have. It takes dataset and column Id that values needed to know as parameters.

In [968]:
def get_potential_splits(data, attribute_ids):
    potential_splits = {}
    for column_index in attribute_ids:  # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        potential_splits[COLUMN_HEADERS[column_index]] = unique_values
    return potential_splits

#### calculate_entropy(data):
Function that calculates the ENTROPHY required to calculate the GAIN value. In the calculation, the ENTROPHY value of the data set is high. This shows us that there is a polarized distribution

In [969]:
def calculate_entropy(data):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)
    probabilities = counts / counts.sum()  # counts = [200 320] 200-negatives, 320-positives
    # print(probabilities)
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

#### calculate_gain(data, index):
Function that returns the GAIN value, which allows us to decide on the next branch of the decision tree. It takes dataset and column index as parameter. In the calculation, we see the GAIN value of the **age** attribute.

In [970]:
# calculate entropy for determined column
def calculate_gain(data, index):
    values, counts = np.unique(data[:, index], return_counts=True)
    total = sum(counts)
    full_entropy = calculate_entropy(data)
    for i in range(len(values)):
        value_data = data[data[:, index] == values[i]]
        value_entropy = calculate_entropy(value_data)
        full_entropy -= (counts[i] / total) * value_entropy
    return full_entropy

#### calculate_best_gain(data, attribute_ids):
Function that returns the best GAIN value in a given dataset and attribute that has this value. It takes dataset and ID list of attributes as parameter. In this example we see the GAIN value = 0.3622... and index 2 = Polyuria

In [971]:
def calculate_best_gain(data, attribute_ids):
    gains = {}
    for id in attribute_ids:
        gain = calculate_gain(data, id)
        gains[id] = gain

    best_gain_key = max(gains, key=gains.get)
    return gains[best_gain_key], best_gain_key

### class Node:
The class in which we create the decision tree <br>
**children** : list of sub branch <br>
**vals** : the value list that attribute has (for age = 1,2,3,4,5 for polyuria = yes, no ... ) <br>
**rule** : rule of desicion tree <br>
**max_gain** : GAIN of that branch <br>
**attribute_name** : Name of attribute (Age,Gender,Polyuria,Polydipsia,sudden weight loss
...) <br>
**attribute_id** : Index of the attribute (for age = 0 , gender = 1, Polyuria = 2 ...) <br>
**chooseLeaf** : In the prune algorithm, twig is changed to leaf and, twig value is set the majority of ”Positive” or ”Negative” as leaf value. For this, we keep which node is in majority, and when twig in leaf is required, the twig vals value is replaced with the value of chooseLeaf.<br>
**ex_node(node)** : For the prune algorithm, we added a new function to the Node class. With the **ex_node function**, we were able to keep the parent node of the current node. This way, if current_accuracy is smaller than last_accuracy, we were able to revert the tree

In [972]:
class Node:
    def __init__(self, attribute_id, attribute_name, vals, rule):
        self.children = []
        self.vals = vals
        self.rule = rule
        self.max_gain = None
        self.attribute_name = attribute_name
        self.attribute_id = attribute_id
        self.chooseLeaf = None
    def ex_node(self, node):
        new_node = Node(node.attribute_id,node.attribute_name,node.vals,node.rule)
        self.children = node.children
        self.vals = node.vals
        self.rule = node.rule
        self.max_gain = node.max_gain
        self.attribute_name = node.attribute_name
        self.attribute_id = node.attribute_id
        self.chooseLeaf = node.chooseLeaf
        return new_node

In [973]:
global POTENTIAL_SPLITS
POTENTIAL_SPLITS = get_potential_splits(df, COLUMN_IDS)

In [974]:
POTENTIAL_SPLITS

{'Age': array([1, 2, 3, 4, 5], dtype=object),
 'Gender': array(['Female', 'Male'], dtype=object),
 'Polyuria': array(['No', 'Yes'], dtype=object),
 'Polydipsia': array(['No', 'Yes'], dtype=object),
 'sudden weight loss': array(['No', 'Yes'], dtype=object),
 'weakness': array(['No', 'Yes'], dtype=object),
 'Polyphagia': array(['No', 'Yes'], dtype=object),
 'Genital thrush': array(['No', 'Yes'], dtype=object),
 'visual blurring': array(['No', 'Yes'], dtype=object),
 'Itching': array(['No', 'Yes'], dtype=object),
 'Irritability': array(['No', 'Yes'], dtype=object),
 'delayed healing': array(['No', 'Yes'], dtype=object),
 'partial paresis': array(['No', 'Yes'], dtype=object),
 'muscle stiffness': array(['No', 'Yes'], dtype=object),
 'Alopecia': array(['No', 'Yes'], dtype=object),
 'Obesity': array(['No', 'Yes'], dtype=object),
 'label': array(['Negative', 'Positive'], dtype=object)}

Unlike the previous decision tree algorithm, a node_list is used to hold all the nodes we have, and before each return, this child_node is added to the node_list. This node_list will then be used to identify nodes with twigs.

In [975]:
def decision_tree_algorithm(data, attribute_ids , rule , node_list):

    if check_purity(data):
        classification = classify_data(data)
        new_rule = str(rule) + classification
        print(new_rule)
        return Node(-1, COLUMN_HEADERS[-1], classification , new_rule)

    if len(attribute_ids[:-1]) == 0:
        targets, counts = np.unique(data[:,-1], return_counts=True)
        index = np.argmax(counts)
        new_rule = str(rule) + " -> " + targets[index]
        print(new_rule)
        return Node(-1, COLUMN_HEADERS[-1], targets[index], new_rule)

    best_attr_val, best_attr_index = calculate_best_gain(data, attribute_ids[:-1])
    potential_values = POTENTIAL_SPLITS[COLUMN_HEADERS[best_attr_index]]

    child_node = Node(best_attr_index, COLUMN_HEADERS[best_attr_index], potential_values , rule)
    child_node.max_gain = best_attr_val
    child_node.rule += COLUMN_HEADERS[best_attr_index]  + " -> "

    for val in potential_values: #yes and no
        val_data = data[data[:,best_attr_index] == val]

        if val_data.shape[0] == 0:
            targets, counts = np.unique(data[:, -1], return_counts=True)
            index = np.argmax(counts)
            new_rule = str(child_node.rule) + str(val) + " -> " + targets[index]
            print(new_rule)
            child_node.children.append(Node(-1, COLUMN_HEADERS[-1], targets[index],new_rule))
        else:
            attribute_ids_cpy = attribute_ids[attribute_ids != best_attr_index]
            new_rule = str(child_node.rule) + str(val) + " -> "
            child_node.children.append(decision_tree_algorithm(val_data, attribute_ids_cpy ,new_rule,node_list))
    
    #to find the majority of ”Positive” or ”Negative” as leaf value
    targets, counts = np.unique(data[:, -1], return_counts=True)
    index = np.argmax(counts)
    chooseLeafValue = targets[index] 
    child_node.chooseLeaf = chooseLeafValue
    
    node_list.append(child_node)

    return child_node

In [976]:
def predict_row_recr(node, row):
    if node.attribute_name == 'label':
        return node.vals
    else:
        index = np.nonzero(node.vals == row[node.attribute_id])[0][0]
        next_node = node.children[index]
        return predict_row_recr(next_node, row)


def predict(root, data):
    predicts = []
    for i in data:
        x = predict_row_recr(root, i)
        predicts.append(x)
    return predicts

#### Prune algorithm 
First, an empty node list is defined. In this way, a list of all nodes added to the tree is kept. Then the tree is run on the test dataset for the last accuracy value and the pre_pruning operation. Then, the algorithm described in the assignment is applied for the prune operation. <br>
A twig list is defined and added to all twigs found in the node_list. Then the node with the lowest gain in the twig list is used for prune operation. First, the found node is copied with the ex_node function. That way, if we need to go back, we don't lose the parent node. Then the content of the twig is changed. The name attibute is set to "label", attribute_id is set to 16, the value of the vals is changed to the value of chooseLeaf, the children in it are deleted and the new rule is defined.
To give an example for the rule,<br>
Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> Alopecia -> -> No -> Age -> <br>rule changed to<br> Polydipsia --> No -> Polyuria -> No -> Gender -> Female -> Alopecia -> No -> Positive. <br>
The predict operation is done with the new tree. current_accuracy is calculated with validate and if current accuracy is not lower than last accuracy, last_accuracy is set to current_accuracy. For the post pruning tree, the tree is saved. If current accuracy is lower than last accuracy, twig node(min_node) will be restored. The loop is terminated. Then, using the post_pruning_tree algorithm, labels are determined for the test dataset and accuracy is calculated.

In [982]:
node_list = []
print("Pre Pruning Tree Rules List:")
tree = decision_tree_algorithm(train, COLUMN_IDS, "", node_list)
predicts = predict(tree, validate)
last_accuracy = accuracy_score(validate[:, -1], predicts)
pre_pruning_tree = tree

#pre pruning tree using
predicts = predict(pre_pruning_tree, test)
acc = accuracy_score(test[:, -1], predicts)
print("Pre pruning " , acc)
print("-------------------------------AFTER PRUNING------------------------------------------------")
post_pruning_tree  = tree
while True:
    twig_list = []
    for parent_node in node_list:
        if len(parent_node.children) != 0:
            twig = True
            for child in parent_node.children:
                if len(child.children) != 0:
                    twig = False
                    break
            if twig:
                twig_list.append(parent_node)

    min_node = twig_list[0]
    for node in twig_list:
        if node.max_gain < min_node.max_gain:
            min_node = node

    ex_node = min_node.ex_node(min_node)
    min_node.rule = min_node.rule.replace(min_node.attribute_name, min_node.chooseLeaf)
    min_node.attribute_name = "label"
    min_node.attribute_id= 16
    min_node.vals = min_node.chooseLeaf
    min_node.children = []


    predicts = predict(tree, validate)
    current_accuracy = 0
    current_accuracy = accuracy_score(validate[:, -1], predicts)
    
    if(current_accuracy < last_accuracy):
        min_node = ex_node.ex_node(ex_node)
        break
    last_accuracy = current_accuracy
    post_pruning_tree = tree
    print("After pruning, this rule change from " , ex_node.rule, "\n To \n" , min_node.rule + "\n")


#post pruning tree using
predicts = predict(post_pruning_tree, test)
current_accuracy = accuracy_score(test[:, -1], predicts)
print("Post pruning " , current_accuracy)


Pre Pruning Tree Rules List:
Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> delayed healing -> No -> visual blurring -> No -> Age -> 1 -> Negative
Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> delayed healing -> No -> visual blurring -> No -> Age -> 2 -> Obesity -> No -> sudden weight loss -> No -> weakness -> No -> Polyphagia -> No -> Genital thrush -> No -> Itching -> No -> Irritability -> No -> partial paresis -> No -> muscle stiffness -> No -> Alopecia -> No ->  -> Negative
Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> delayed healing -> No -> visual blurring -> No -> Age -> 2 -> Obesity -> No -> sudden weight loss -> No -> weakness -> No -> Polyphagia -> No -> Genital thrush -> No -> Itching -> No -> Irritability -> No -> partial paresis -> No -> muscle stiffness -> No -> Alopecia -> Yes -> Negative
Polydipsia -> No -> Polyuria -> No -> Gender -> Female -> delayed healing -> No -> visual blurring -> No -> Age -> 2 -> Obesity -> No -> sudden weigh

Pre_pruning accuracy : 0.9038461538461539 <br>
Post pruning accuarcy : 0.9423076923076923

In post pruning, full tree is generated and then the non-significant branches are removed. Calculating accuracy is performed at every step to check whether addition of the new branch leads to increase in accuracy. If not the branch is converted to leaf node. Pruning can not only significantly reduce the size but also improve the classification accuracy of unseen objects. <br>

The output above lists the rules that change during pruning. The difference between the two models is that the low-gain twigs in the first model, consisting only of leaf nodes, are replaced with labels. As can be seen in this example, although sometimes better accuracy is obtained from the model, better accuracy is not always obtained. The foremost of our reasons for performing this operation is to prevent overfitting. For this, the model is constantly updated with the validation dataset and branches that may cause overfitting are trimmed. Finally, it is tested with the test dataset. Along with pruning, the dataset can also be enlarged for a better result. Better accuracy could be achieved with a larger and more diverse data set.